In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType

spark = SparkSession.builder.getOrCreate()
eqDF = spark.read.option('header','true').option('delimiter','\t').option('inferSchema','true').csv('datasets/EarthquakeDataset-latest.txt')
eqDF = eqDF.select('Date','Origin Time','Latitude','Longitude','xM','Location')

def setRegion(lat,lng):
    region = ""
    if lat >= 39:
        if lng <= 31:
            region = "Region 1"
        elif lng <= 36:
            region = "Region 2"
        elif lng <= 41:
            region = "Region 3"
        else:
            region = "Region 4"
    else:
        if lng <= 31:
            region = "Region 5"
        elif lng <= 36:
            region = "Region 6"
        elif lng <= 41:
            region = "Region 7"
        else:
            region = "Region 8"
    return region

setRegionUDF = UserDefinedFunction(setRegion,StringType())
#eqDF = eqDF.filter(eqDF['xM']>5.0)
eqDF = eqDF[eqDF['xM']>5.0]
eqDF = eqDF.withColumn('Region',setRegionUDF('Latitude','Longitude'))
eqDF = eqDF.groupBy('Region').count()
eqDF = eqDF.sort('count',ascending=False)
eqDF.show()

+--------+-----+
|  Region|count|
+--------+-----+
|Region 5|  412|
|Region 1|  167|
|Region 4|   90|
|Region 2|   77|
|Region 3|   76|
|Region 7|   68|
|Region 8|   68|
|Region 6|   59|
+--------+-----+

